# Simon's Algorithm
Suppose we’re given a function $f:\{0,1\}^n \rightarrow \{0,1\}^n$ that maps bit strings to bit strings along with the promise that
$$\forall x,y \in \{0,1\}^n, \quad f(x) = f(y) \iff x=y\oplus s,$$
for some unknown $n$-bit string $s \in \{0,1\}^n$, and where $\oplus$ means bitwise addition modulo 2.

The goal of Simon's problem is to determine if $f$ is one-to-one, or two-to-one, or equivalently to find the secret string $s$.

# References 
[1] Wikipedia: [Simon's Problem](https://en.wikipedia.org/wiki/Simon%27s_problem)

[2] Amazon Braket's github repository: [Detailed example notebook](https://github.com/aws/amazon-braket-examples/blob/main/examples/advanced_circuits_algorithms/Simons_Algorithm/Simons_Algorithm.ipynb)

# Run on a local simulator

In [1]:
import numpy as np
import numpy.random as npr

from simons_algorithm import simons_oracle, simons_circuit, submit_simons_task, process_simons_results
from braket.aws import AwsDevice
from braket.devices import LocalSimulator
from braket.tracking import Tracker

In [2]:
tracker = Tracker().start()

n = 5
s = "".join(npr.choice(["0","1"], size=n))

oracle = simons_oracle(s)

# Print the circuits

In [3]:
print(simons_circuit(oracle))


T  : |0|    1    | 2 |3|4|5|
                            
q0 : -H-C---------H---------
        |                   
q1 : -H-|-C-------C---C-C-H-
        | |       |   | |   
q2 : -H-|-|-C-----|-H-|-|---
        | | |     |   | |   
q3 : -H-|-|-|-C---|-H-|-|---
        | | | |   |   | |   
q4 : -H-|-|-|-|-C-|-H-|-|---
        | | | | | |   | |   
q5 : ---X-|-|-|-|-|---|-|---
          | | | | |   | |   
q6 : -----X-|-|-|-X---|-|---
            | | |     | |   
q7 : -------X-|-|-----X-|---
              | |       |   
q8 : ---------X-|-------|---
                |       |   
q9 : -----------X-------X---

T  : |0|    1    | 2 |3|4|5|


In [4]:
local_simulator = LocalSimulator()

task = submit_simons_task(oracle, local_simulator)

In [5]:
secret_revealed = process_simons_results(task)
print("Result string:", secret_revealed)
print("Secret string", s)

Result string: 01101
Secret string 01101


# Run on a QPU

Here we run the Simon's algorithm on the Rigetti Aspen M-2 device. The cost to run this experiment is \$0.3 per task and \$0.00035 per shot. When submitting a circuit, we specify a number of shots that is twice the number of qubits (i.e. four times the length of the secret string). For a string of 5 characters, the total cost is \$0.31 USD.

In [6]:
aspen_m2 = AwsDevice("arn:aws:braket:us-west-1::device/qpu/rigetti/Aspen-M-2")
task = submit_simons_task(oracle, aspen_m2)

In [7]:
secret_revealed = process_simons_results(task)
print("Result string:", secret_revealed)
print("Secret string", s)

Result string: 00000
Secret string 01101


In [8]:
print(
    f"Estimated cost to run this example: {tracker.qpu_tasks_cost() + tracker.simulator_tasks_cost():.2f} USD"
)

Estimated cost to run this example: 0.31 USD


Note: Charges shown are estimates based on your Amazon Braket simulator and quantum processing unit (QPU) task usage. Estimated charges shown may differ from your actual charges. Estimated charges do not factor in any discounts or credits, and you may experience additional charges based on your use of other services such as Amazon Elastic Compute Cloud (Amazon EC2).